<a href="https://colab.research.google.com/github/yenchunR/NLP_HW4/blob/master/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Module

In [41]:
!pip3 install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 5.3 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486150 sha256=6b1af0cef67f492f02aea2a8075610d522d9322aef2687e6bcd0d87a6a1dc831
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [91]:
!pip3 install gensim

# 將Colab連結Drive

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 讀取所有檔案路徑

In [64]:
import pandas as pd
import os

# 把列出資料夾的程式碼寫成一個函式
filePath = []
def show_folder_content(folder_path):

    folder_content = os.listdir(folder_path)
    for item in folder_content:
        if os.path.isdir(folder_path + '/' + item):
            # 呼叫自己處理這個子資料夾
            show_folder_content(folder_path + '/' + item)
        elif os.path.isfile(folder_path + '/' + item):
            filePath.append(folder_path + '/' + item)
        else:
            print('無法辨識：' + item)


# 顯示指定資料夾的內容
target_folder = '/content/drive/My Drive/wiki_zh'
show_folder_content(target_folder)
print("Finish", len(filePath))


Finish 1279


# jieba 斷詞

In [ ]:
# coding=utf-8
from opencc import OpenCC
import jieba 
import json
import re

dictionaryPath = '/content/drive/My Drive/wiki_zh/DriveUploader/dict.txt.big'
jieba.set_dictionary(dictionaryPath) ##加入繁體中文

# Initial
cc = OpenCC('s2t')

# 讀取停用詞
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

def LoadToSeg(filepath, fW):
    with open(filepath, 'r') as obj:
        for line in obj.readlines():
            dic = json.loads(line)
            temp_text = re.findall('[\u4e00-\u9fa5]', cc.convert(dic['text']))
            text = ''.join(temp_text)
            tags = jieba.lcut(text, cut_all=False)
            ThisLine = ''
            for tag in tags:
                if tag not in stopwords:  
                    if tag != '\t' and tag != '\n' and tag != '' and len(tag) > 1:  
                        ThisLine += tag  
                        ThisLine += " "   #再次組合成【帶空格】的串
            fW.write((ThisLine+"\n").encode('utf-8'))
            
stopwords = stopwordslist('/content/drive/My Drive/wiki_zh/DriveUploader/StopWords.txt')  # 這裏加載停用詞的路徑 
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    fW.truncate(0)
    index = 0
    for path in filePath:
        LoadToSeg(path, fW)
        index += 1
        if index % 100 == 0:
            print(index)

print("Finish")

# word2vec 轉成高維空間向量

In [66]:
from gensim.models import word2vec
# jieba分詞轉word2vec向量
fileSegWordDonePath ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.txt'
fileSegWordDonePath2 ='/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.model'

# Settings
sentences = word2vec.LineSentence(fileSegWordDonePath)
model = word2vec.Word2Vec(sentences, seed = 666, sg = 0, window = 10, min_count = 1, workers = 8)
model.save(fileSegWordDonePath2)
print("Finish")

Finish


# Result

In [2]:
# -*- coding: utf-8 -*-# -*- coding: utf-8 -*-
import sys
from gensim.models import word2vec 
from gensim import models

def main():   
    a = input("input:") 
    fileSegWordDonePath2 = '/content/drive/My Drive/wiki_zh/DriveUploader/corpusSegDone.model'
    model = models.Word2Vec.load(fileSegWordDonePath2)
    try:
        res = model.most_similar(a, topn=20)
        for item in res: 
            print(str(item[0]) + ':' + str(item[1]))
    except KeyError:
        print(a + " is not in the vocabulary")

if __name__=='__main__':
    main()

input:餅乾


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


薯條:0.8262001872062683
糖果:0.8244751691818237
奶油:0.822588324546814
果汁:0.8099068403244019
薯片:0.8083598613739014
軟糖:0.8079686760902405
巧克力:0.8006643056869507
零食:0.7975587844848633
果醬:0.7928873300552368
冰淇淋:0.7911243438720703
雞塊:0.7849758863449097
甜點:0.7848320007324219
雞尾酒:0.7830371856689453
牛排:0.7809081673622131
蛋糕:0.7798120975494385
玉米片:0.7787021398544312
芥末:0.7785236835479736
吐司:0.7784594893455505
蘋果派:0.7771775722503662
焦糖:0.7768616676330566
